In [1]:
import tqdm
import pandas as pd
import os
import sys

In [2]:
parent_dir = os.path.abspath(os.path.join(os.getcwd(), os.pardir))
sys.path.append(parent_dir)

from predict.prediction_models import (
    DensePredictor,
    LayerNormPredictor,
    AttentionPredictor,
    EmbeddingPredictor
)

In [3]:
dense_predictor = DensePredictor(with_features=True)
layer_norm_predictor = LayerNormPredictor()
attention_predictor = AttentionPredictor(with_features=True)
embedding_predictor = EmbeddingPredictor(with_features=True)

In [4]:
dense_wo_features_predictor = DensePredictor(with_features=False)
layer_norm_wo_feaures_predictor = LayerNormPredictor()
attention_wo_feaures_predictor = AttentionPredictor(with_features=False)
embedding_wo_feaures_predictor = EmbeddingPredictor(with_features=False)

In [5]:
gpu_name = 'V100'

In [6]:
# Defining BERT Base model layer specs
batch_size = 64
seq_len = 128
vocab_size = 30522
embed_dim = 768
num_heads = 12
ffn_dim = 3072
num_layers = 12

embedding_layers = {}
attention_layers = {}
dense_layers = {}
layernorm_layers = {}

embedding_layers['embedding'] = {
    'batchsize': batch_size,
    'seq_len': seq_len,
    'vocab_size': vocab_size,
    'embed_dim': embed_dim,
    'optimizer': 'Adam'
}
embedding_layers['positional_embedding'] = {
    'batchsize': batch_size,
    'seq_len': seq_len,
    'vocab_size': seq_len,
    'embed_dim': embed_dim,
    'optimizer': 'Adam'
}

for i in range(num_layers):
    attention_layers[f'attention_{i}'] = {
        'batchsize': batch_size,
        'seq_len': seq_len,
        'num_heads': num_heads,
        'embed_dim': embed_dim,
        'optimizer': 'Adam'
    }
    layernorm_layers[f'layernorm_{i}_1'] = {
        'batchsize': batch_size,
        'seq_len': seq_len,
        'embed_dim': embed_dim,
        'optimizer': 'Adam'
    }
    layernorm_layers[f'layernorm_{i}_2'] = {
        'batchsize': batch_size,
        'seq_len': seq_len,
        'embed_dim': embed_dim,
        'optimizer': 'Adam'
    }
    dense_layers[f'dense_{i}_1'] = {
        'batchsize': batch_size,
        'dim_input': embed_dim,
        'dim_output': ffn_dim,
        'activation_fct': 'ReLU',
        'optimizer': 'Adam'
    }
    dense_layers[f'dense_{i}_2'] = {
        'batchsize': batch_size,
        'dim_input': ffn_dim,
        'dim_output': embed_dim,
        'activation_fct': 'None',
        'optimizer': 'Adam'
    }
dense_layers['dense_final'] = {
    'batchsize': batch_size,
    'dim_input': embed_dim,
    'dim_output': vocab_size,
    'activation_fct': 'None',
    'optimizer': 'Adam'
}

In [7]:
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
import time

dataset_size = 1024

# Check if GPU is available
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Define a simulated dataset for BERT-style training
class RandomTextDataset(Dataset):
    def __init__(self, vocab_size, sequence_length, dataset_size):
        self.vocab_size = vocab_size
        self.sequence_length = sequence_length
        self.dataset_size = dataset_size

    def __len__(self):
        return self.dataset_size

    def __getitem__(self, idx):
        input_ids = torch.randint(0, self.vocab_size, (self.sequence_length,))
        labels = torch.randint(0, self.vocab_size, (self.sequence_length,))
        return input_ids, labels

# Initialize the dataset and DataLoader

train_dataset = RandomTextDataset(vocab_size, seq_len, dataset_size)
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)

# Define BERT-base model with layer-by-layer transformer blocks
class BERTBaseLayerByLayer(nn.Module):
    def __init__(self, vocab_size, embed_dim, num_heads, ffn_dim, num_layers, sequence_length):
        super(BERTBaseLayerByLayer, self).__init__()

        # Embedding layers
        self.token_embedding = nn.Embedding(vocab_size, embed_dim)
        self.position_embedding = nn.Embedding(sequence_length, embed_dim)
        
        # Transformer layers
        self.attention_layers = nn.ModuleList()
        self.ffn_layers = nn.ModuleList()
        self.norm_layers_1 = nn.ModuleList()
        self.norm_layers_2 = nn.ModuleList()

        for _ in range(num_layers):
            self.attention_layers.append(nn.MultiheadAttention(embed_dim=embed_dim, num_heads=num_heads, batch_first=True))
            self.ffn_layers.append(nn.Sequential(
                nn.Linear(embed_dim, ffn_dim),
                nn.ReLU(),
                nn.Linear(ffn_dim, embed_dim)
            ))
            self.norm_layers_1.append(nn.LayerNorm(embed_dim))
            self.norm_layers_2.append(nn.LayerNorm(embed_dim))
        
        # Final output layer
        self.output_layer = nn.Linear(embed_dim, vocab_size)
    
    def forward(self, x):
        position_ids = torch.arange(seq_len, device=x.device).unsqueeze(0).expand(x.size(0), -1)
        x = self.token_embedding(x) + self.position_embedding(position_ids)

        for i in range(num_layers):
            attn_output, _ = self.attention_layers[i](x, x, x)
            x = self.norm_layers_1[i](x + attn_output)
            
            ffn_output = self.ffn_layers[i](x)
            x = self.norm_layers_2[i](x + ffn_output)

        return self.output_layer(x)

# Initialize the model and move it to the GPU
model = BERTBaseLayerByLayer(vocab_size, embed_dim, num_heads, ffn_dim, num_layers, seq_len).to(device)

epochs = 100

# Define a loss function and optimizer
loss_fn = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=1e-4)

# Training loop
start_time = time.time()
for epoch in tqdm.tqdm(range(epochs)):
    
    epoch_loss = 0.0

    for input_data, labels in train_loader:
        input_data, labels = input_data.to(device), labels.to(device)

        # Forward pass
        optimizer.zero_grad()
        outputs = model(input_data)
        
        # Reshape for loss calculation
        outputs = outputs.view(-1, vocab_size)
        labels = labels.view(-1)
        
        # Compute loss
        loss = loss_fn(outputs, labels)
        
        # Backward pass and optimization
        loss.backward()
        optimizer.step()
        
        epoch_loss += loss.item()

    print(f"Epoch [{epoch+1}/{epochs}] - Loss: {epoch_loss / len(train_loader):.4f}")
end_time = time.time()
print("Training complete.")


/home/ubuntu/miniconda/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
  1%|          | 1/100 [00:08<13:55,  8.44s/it]

Epoch [1/100] - Loss: 10.4700


  2%|▏         | 2/100 [00:16<13:31,  8.29s/it]

Epoch [2/100] - Loss: 10.4197


  3%|▎         | 3/100 [00:24<13:19,  8.24s/it]

...

Epoch [98/100] - Loss: 10.3367


 99%|█████████▉| 99/100 [13:33<00:08,  8.20s/it]

Epoch [99/100] - Loss: 10.3369


100%|██████████| 100/100 [13:41<00:00,  8.22s/it]

Epoch [100/100] - Loss: 10.3373
Training complete.


In [8]:
time_elapsed = end_time - start_time # Calculate the total time taken for training in seconds
print(f"Total time taken for training BERT: {time_elapsed:.2f} seconds")

Total time taken for training BERT: 821.55 seconds


In [9]:
time_for_one_epoch = time_elapsed / epochs
print(f"Average time taken for one epoch of BERT: {time_for_one_epoch:.2f} seconds")

Average time taken for one epoch of BERT: 8.22 seconds


In [10]:
gpu_specs = pd.read_csv('../gpus/GPUs.csv')
gpu_specs

,GPU,Provisioning,Base Clock (MHz),Boost Clock (MHz),Memory Clock (MHz),Memory (GB),Memory Type,Memory Bus (bit),GPU Memory Bandwidth (GB/s),Bus,...,TMUs,ROPs,SM,TC,RT,PR,TR,FP16,FP32,FP64
0,L4,Cloud,795,2040,1563,24,GDDR6,192,300,PCIe 4.0,...,240,80,60,240,60,163,490,30290,30290,473
1,P4,Cloud,886,1114,1502,8,GDDR5,256,192,PCIe 3.0,...,160,64,20,0,0,71,178,89,5700,178
2,P100,Cloud,1190,1329,715,16,HBM2,4096,732,PCIe 3.0,...,224,96,56,0,0,127,297,19050,9526,4763
3,RTX4090,Local,2235,2520,1313,24,GDDR6X,384,1001,PCIe 4.0,...,512,176,128,512,128,443,1290,82580,82580,1290
4,RTXA4000,Local,735,1560,1750,16,GDDR6,256,448,PCIe 4.0,...,192,96,48,192,48,150,300,19170,19170,300
5,T4,Cloud,585,1590,1250,16,GDDR6,256,320,PCIe 3.0,...,160,64,40,320,40,101,254,65130,8141,254
6,V100,Cloud,1245,1380,876,16,HBM2,4096,897,PCIe 3.0,...,320,128,80,640,0,176,441,28260,14130,7066


In [11]:
dense_layers_features = dense_layers
embedding_layers_features = embedding_layers
attention_layers_features = attention_layers
layernorm_layers_features = layernorm_layers

In [12]:
gpu = gpu_specs[gpu_specs['GPU'] == gpu_name]
gpu = gpu.squeeze()
gpu = pd.DataFrame([gpu] * len(dense_layers_features))

dense_layers_features = pd.DataFrame.from_dict(dense_layers_features, orient='index')
dense_layers_features = pd.concat([dense_layers_features.reset_index(drop=True), gpu.reset_index(drop=True)], axis=1)


gpu = gpu_specs[gpu_specs['GPU'] == gpu_name]
gpu = gpu.squeeze()
gpu = pd.DataFrame([gpu] * len(embedding_layers_features))

embedding_layers_features = pd.DataFrame.from_dict(embedding_layers_features, orient='index')
embedding_layers_features = pd.concat([embedding_layers_features.reset_index(drop=True), gpu.reset_index(drop=True)], axis=1)


gpu = gpu_specs[gpu_specs['GPU'] == gpu_name]
gpu = gpu.squeeze()
gpu = pd.DataFrame([gpu] * len(attention_layers_features))

attention_layers_features = pd.DataFrame.from_dict(attention_layers_features, orient='index')
attention_layers_features = pd.concat([attention_layers_features.reset_index(drop=True), gpu.reset_index(drop=True)], axis=1)

gpu = gpu_specs[gpu_specs['GPU'] == gpu_name]
gpu = gpu.squeeze()
gpu = pd.DataFrame([gpu] * len(layernorm_layers_features))

layernorm_layers_features = pd.DataFrame.from_dict(layernorm_layers_features, orient='index')
layernorm_layers_features = pd.concat([layernorm_layers_features.reset_index(drop=True), gpu.reset_index(drop=True)], axis=1)

In [13]:
dense_layers_features['optimizer'] = 4
dense_layers_features['precision'] = 32
dense_layers_features['activation_fct'] = dense_layers_features['activation_fct'].replace({'ReLU': 1, 'None': 0})

/tmp/ipykernel_576551/136968382.py:3: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  dense_layers_features['activation_fct'] = dense_layers_features['activation_fct'].replace({'ReLU': 1, 'None': 0})


In [14]:
dense_layers_features

,batchsize,dim_input,dim_output,activation_fct,optimizer,GPU,Provisioning,Base Clock (MHz),Boost Clock (MHz),Memory Clock (MHz),...,ROPs,SM,TC,RT,PR,TR,FP16,FP32,FP64,precision
0,64,768,3072,1,4,V100,Cloud,1245,1380,876,...,128,80,640,0,176,441,28260,14130,7066,32
1,64,3072,768,0,4,V100,Cloud,1245,1380,876,...,128,80,640,0,176,441,28260,14130,7066,32
2,64,768,3072,1,4,V100,Cloud,1245,1380,876,...,128,80,640,0,176,441,28260,14130,7066,32
3,64,3072,768,0,4,V100,Cloud,1245,1380,876,...,128,80,640,0,176,441,28260,14130,7066,32
4,64,768,3072,1,4,V100,Cloud,1245,1380,876,...,128,80,640,0,176,441,28260,14130,7066,32
5,64,3072,768,0,4,V100,Cloud,1245,1380,876,...,128,80,640,0,176,441,28260,14130,7066,32
6,64,768,3072,1,4,V100,Cloud,1245,1380,876,...,128,80,640,0,176,441,28260,14130,7066,32
7,64,3072,768,0,4,V100,Cloud,1245,1380,876,...,128,80,640,0,176,441,28260,14130,7066,32
8,64,768,3072,1,4,V100,Cloud,1245,1380,876,...,128,80,640,0,176,441,28260,14130,7066,32
9,64,3072,768,0,4,V100,Cloud,1245,1380,876,...,128,80,640,0,176,441,28260,14130,7066,32


In [15]:
embedding_layers_features['optimizer'] = 4
embedding_layers_features['precision'] = 32

In [16]:
embedding_layers_features

,batchsize,seq_len,vocab_size,embed_dim,optimizer,GPU,Provisioning,Base Clock (MHz),Boost Clock (MHz),Memory Clock (MHz),...,ROPs,SM,TC,RT,PR,TR,FP16,FP32,FP64,precision
0,64,128,30522,768,4,V100,Cloud,1245,1380,876,...,128,80,640,0,176,441,28260,14130,7066,32
1,64,128,128,768,4,V100,Cloud,1245,1380,876,...,128,80,640,0,176,441,28260,14130,7066,32


In [17]:
attention_layers_features['optimizer'] = 4
attention_layers_features['precision'] = 32

In [18]:
attention_layers_features

,batchsize,seq_len,num_heads,embed_dim,optimizer,GPU,Provisioning,Base Clock (MHz),Boost Clock (MHz),Memory Clock (MHz),...,ROPs,SM,TC,RT,PR,TR,FP16,FP32,FP64,precision
0,64,128,12,768,4,V100,Cloud,1245,1380,876,...,128,80,640,0,176,441,28260,14130,7066,32
1,64,128,12,768,4,V100,Cloud,1245,1380,876,...,128,80,640,0,176,441,28260,14130,7066,32
2,64,128,12,768,4,V100,Cloud,1245,1380,876,...,128,80,640,0,176,441,28260,14130,7066,32
3,64,128,12,768,4,V100,Cloud,1245,1380,876,...,128,80,640,0,176,441,28260,14130,7066,32
4,64,128,12,768,4,V100,Cloud,1245,1380,876,...,128,80,640,0,176,441,28260,14130,7066,32
5,64,128,12,768,4,V100,Cloud,1245,1380,876,...,128,80,640,0,176,441,28260,14130,7066,32
6,64,128,12,768,4,V100,Cloud,1245,1380,876,...,128,80,640,0,176,441,28260,14130,7066,32
7,64,128,12,768,4,V100,Cloud,1245,1380,876,...,128,80,640,0,176,441,28260,14130,7066,32
8,64,128,12,768,4,V100,Cloud,1245,1380,876,...,128,80,640,0,176,441,28260,14130,7066,32
9,64,128,12,768,4,V100,Cloud,1245,1380,876,...,128,80,640,0,176,441,28260,14130,7066,32


In [19]:
layernorm_layers_features['precision'] = 32
layernorm_layers_features['optimizer'] = 4

In [20]:
layernorm_layers_features

,batchsize,seq_len,embed_dim,optimizer,GPU,Provisioning,Base Clock (MHz),Boost Clock (MHz),Memory Clock (MHz),Memory (GB),...,ROPs,SM,TC,RT,PR,TR,FP16,FP32,FP64,precision
0,64,128,768,4,V100,Cloud,1245,1380,876,16,...,128,80,640,0,176,441,28260,14130,7066,32
1,64,128,768,4,V100,Cloud,1245,1380,876,16,...,128,80,640,0,176,441,28260,14130,7066,32
2,64,128,768,4,V100,Cloud,1245,1380,876,16,...,128,80,640,0,176,441,28260,14130,7066,32
3,64,128,768,4,V100,Cloud,1245,1380,876,16,...,128,80,640,0,176,441,28260,14130,7066,32
4,64,128,768,4,V100,Cloud,1245,1380,876,16,...,128,80,640,0,176,441,28260,14130,7066,32
5,64,128,768,4,V100,Cloud,1245,1380,876,16,...,128,80,640,0,176,441,28260,14130,7066,32
6,64,128,768,4,V100,Cloud,1245,1380,876,16,...,128,80,640,0,176,441,28260,14130,7066,32
7,64,128,768,4,V100,Cloud,1245,1380,876,16,...,128,80,640,0,176,441,28260,14130,7066,32
8,64,128,768,4,V100,Cloud,1245,1380,876,16,...,128,80,640,0,176,441,28260,14130,7066,32
9,64,128,768,4,V100,Cloud,1245,1380,876,16,...,128,80,640,0,176,441,28260,14130,7066,32


In [21]:
from predict.features import (
    PreprocessDenseFeatures,
    PreprocessAttentionFeatures,
    PreprocessEmbeddingFeatures,
    PreprocessLayerNormFeatures,
)

dense_features = PreprocessDenseFeatures(dense_layers_features, include_additional_features=True).features
attention_features = PreprocessAttentionFeatures(attention_layers_features, include_additional_features=True).features
embedding_features = PreprocessEmbeddingFeatures(embedding_layers_features, include_additional_features=True).features
layernorm_features = PreprocessLayerNormFeatures(layernorm_layers_features, include_additional_features=True).features

dense_features_less = PreprocessDenseFeatures(dense_layers_features, include_additional_features=False).features
attention_features_less = PreprocessAttentionFeatures(attention_layers_features, include_additional_features=False).features
embedding_features_less = PreprocessEmbeddingFeatures(embedding_layers_features, include_additional_features=False).features
layernorm_features_less = PreprocessLayerNormFeatures(layernorm_layers_features, include_additional_features=False).features

/home/ubuntu/dl-training-time-prediction/predict/features.py:387: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value '['Adam' 'Adam' 'Adam' 'Adam' 'Adam' 'Adam' 'Adam' 'Adam' 'Adam' 'Adam'
 'Adam' 'Adam' 'Adam' 'Adam' 'Adam' 'Adam' 'Adam' 'Adam' 'Adam' 'Adam'
 'Adam' 'Adam' 'Adam' 'Adam' 'Adam']' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  self.features.loc[:, 'optimizer'] = self.features['optimizer'].map({0:'None',
/home/ubuntu/dl-training-time-prediction/predict/features.py:394: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value '['relu' 'None' 'relu' 'None' 'relu' 'None' 'relu' 'None' 'relu' 'None'
 'relu' 'None' 'relu' 'None' 'relu' 'None' 'relu' 'None' 'relu' 'None'
 'relu' 'None' 'relu' 'None' 'None']' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  self.features.lo

In [22]:
dense_features = dense_features.drop(columns=['GPU'])
attention_features = attention_features.drop(columns=['GPU'])
embedding_features = embedding_features.drop(columns=['GPU'])
layernorm_features = layernorm_features.drop(columns=['GPU'])

dense_features_less = dense_features_less.drop(columns=['GPU'])
attention_features_less = attention_features_less.drop(columns=['GPU'])
embedding_features_less = embedding_features_less.drop(columns=['GPU'])
layernorm_features_less = layernorm_features_less.drop(columns=['GPU'])

In [23]:
dense_features

,Base Clock (MHz),Boost Clock (MHz),Bus_PCIe 3.0,Bus_PCIe 4.0,Cores,FP32,GPU Memory Bandwidth (GB/s),Memory (GB),Memory Bus (bit),Memory Clock (MHz),...,memory_out,memory_total,memory_weights,optimizer_Adadelta,optimizer_Adagrad,optimizer_Adam,optimizer_None,optimizer_RMSProp,optimizer_SGD,precision
0,1245,1380,1.0,0.0,5120,14130,897,16,4096,876,...,196608,5210112,2359296,0.0,0.0,1.0,0.0,0.0,0.0,32
1,1245,1380,1.0,0.0,5120,14130,897,16,4096,876,...,49152,5210112,2359296,0.0,0.0,1.0,0.0,0.0,0.0,32
2,1245,1380,1.0,0.0,5120,14130,897,16,4096,876,...,196608,5210112,2359296,0.0,0.0,1.0,0.0,0.0,0.0,32
3,1245,1380,1.0,0.0,5120,14130,897,16,4096,876,...,49152,5210112,2359296,0.0,0.0,1.0,0.0,0.0,0.0,32
4,1245,1380,1.0,0.0,5120,14130,897,16,4096,876,...,196608,5210112,2359296,0.0,0.0,1.0,0.0,0.0,0.0,32
5,1245,1380,1.0,0.0,5120,14130,897,16,4096,876,...,49152,5210112,2359296,0.0,0.0,1.0,0.0,0.0,0.0,32
6,1245,1380,1.0,0.0,5120,14130,897,16,4096,876,...,196608,5210112,2359296,0.0,0.0,1.0,0.0,0.0,0.0,32
7,1245,1380,1.0,0.0,5120,14130,897,16,4096,876,...,49152,5210112,2359296,0.0,0.0,1.0,0.0,0.0,0.0,32
8,1245,1380,1.0,0.0,5120,14130,897,16,4096,876,...,196608,5210112,2359296,0.0,0.0,1.0,0.0,0.0,0.0,32
9,1245,1380,1.0,0.0,5120,14130,897,16,4096,876,...,49152,5210112,2359296,0.0,0.0,1.0,0.0,0.0,0.0,32


In [24]:
attention_features

,Base Clock (MHz),Boost Clock (MHz),Bus_PCIe 3.0,Bus_PCIe 4.0,Cores,FP32,GPU Memory Bandwidth (GB/s),Memory (GB),Memory Bus (bit),Memory Clock (MHz),...,memory_total,num_heads,optimizer_Adadelta,optimizer_Adagrad,optimizer_Adam,optimizer_None,optimizer_RMSProp,optimizer_SGD,precision,seq_len
0,1245,1380,1.0,0.0,5120,14130,897,16,4096,876,...,78643200,12,0.0,0.0,1.0,0.0,0.0,0.0,32,128
1,1245,1380,1.0,0.0,5120,14130,897,16,4096,876,...,78643200,12,0.0,0.0,1.0,0.0,0.0,0.0,32,128
2,1245,1380,1.0,0.0,5120,14130,897,16,4096,876,...,78643200,12,0.0,0.0,1.0,0.0,0.0,0.0,32,128
3,1245,1380,1.0,0.0,5120,14130,897,16,4096,876,...,78643200,12,0.0,0.0,1.0,0.0,0.0,0.0,32,128
4,1245,1380,1.0,0.0,5120,14130,897,16,4096,876,...,78643200,12,0.0,0.0,1.0,0.0,0.0,0.0,32,128
5,1245,1380,1.0,0.0,5120,14130,897,16,4096,876,...,78643200,12,0.0,0.0,1.0,0.0,0.0,0.0,32,128
6,1245,1380,1.0,0.0,5120,14130,897,16,4096,876,...,78643200,12,0.0,0.0,1.0,0.0,0.0,0.0,32,128
7,1245,1380,1.0,0.0,5120,14130,897,16,4096,876,...,78643200,12,0.0,0.0,1.0,0.0,0.0,0.0,32,128
8,1245,1380,1.0,0.0,5120,14130,897,16,4096,876,...,78643200,12,0.0,0.0,1.0,0.0,0.0,0.0,32,128
9,1245,1380,1.0,0.0,5120,14130,897,16,4096,876,...,78643200,12,0.0,0.0,1.0,0.0,0.0,0.0,32,128


In [25]:
embedding_features

,Base Clock (MHz),Boost Clock (MHz),Bus_PCIe 3.0,Bus_PCIe 4.0,Cores,FP32,GPU Memory Bandwidth (GB/s),Memory (GB),Memory Bus (bit),Memory Clock (MHz),...,memory_total,optimizer_Adadelta,optimizer_Adagrad,optimizer_Adam,optimizer_None,optimizer_RMSProp,optimizer_SGD,precision,seq_len,vocab_size
0,1245,1380,1.0,0.0,5120,14130,897,16,4096,876,...,59464704,0.0,0.0,1.0,0.0,0.0,0.0,32,128,30522
1,1245,1380,1.0,0.0,5120,14130,897,16,4096,876,...,12779520,0.0,0.0,1.0,0.0,0.0,0.0,32,128,128


In [26]:
layernorm_features

,Base Clock (MHz),Boost Clock (MHz),Bus_PCIe 3.0,Bus_PCIe 4.0,Cores,FP32,GPU Memory Bandwidth (GB/s),Memory (GB),Memory Bus (bit),Memory Clock (MHz),...,batchsize,embed_dim,optimizer_Adadelta,optimizer_Adagrad,optimizer_Adam,optimizer_None,optimizer_RMSProp,optimizer_SGD,precision,seq_len
0,1245,1380,1.0,0.0,5120,14130,897,16,4096,876,...,64,768,0.0,0.0,1.0,0.0,0.0,0.0,32,128
1,1245,1380,1.0,0.0,5120,14130,897,16,4096,876,...,64,768,0.0,0.0,1.0,0.0,0.0,0.0,32,128
2,1245,1380,1.0,0.0,5120,14130,897,16,4096,876,...,64,768,0.0,0.0,1.0,0.0,0.0,0.0,32,128
3,1245,1380,1.0,0.0,5120,14130,897,16,4096,876,...,64,768,0.0,0.0,1.0,0.0,0.0,0.0,32,128
4,1245,1380,1.0,0.0,5120,14130,897,16,4096,876,...,64,768,0.0,0.0,1.0,0.0,0.0,0.0,32,128
5,1245,1380,1.0,0.0,5120,14130,897,16,4096,876,...,64,768,0.0,0.0,1.0,0.0,0.0,0.0,32,128
6,1245,1380,1.0,0.0,5120,14130,897,16,4096,876,...,64,768,0.0,0.0,1.0,0.0,0.0,0.0,32,128
7,1245,1380,1.0,0.0,5120,14130,897,16,4096,876,...,64,768,0.0,0.0,1.0,0.0,0.0,0.0,32,128
8,1245,1380,1.0,0.0,5120,14130,897,16,4096,876,...,64,768,0.0,0.0,1.0,0.0,0.0,0.0,32,128
9,1245,1380,1.0,0.0,5120,14130,897,16,4096,876,...,64,768,0.0,0.0,1.0,0.0,0.0,0.0,32,128


In [27]:
predicted_dense = dense_predictor.predict(dense_features)
predicted_attention = attention_predictor.predict(attention_features)
predicted_layernorm = layer_norm_predictor.predict(layernorm_features)
predicted_embedding = embedding_predictor.predict(embedding_features)

predicted_dense = sum(predicted_dense * (dataset_size / batch_size)) / 1000
predicted_attention = sum(predicted_attention * (dataset_size / batch_size)) / 1000
predicted_embedding = sum(predicted_embedding * (dataset_size / batch_size)) / 1000
predicted_layernorm = sum(predicted_layernorm * (dataset_size / batch_size)) / 1000

predicted_e2e = predicted_dense + predicted_attention + predicted_embedding + predicted_layernorm
predicted_e2e = predicted_e2e.item()

In [28]:
predicted_dense_less = dense_wo_features_predictor.predict(dense_features_less)
predicted_attention_less = attention_wo_feaures_predictor.predict(attention_features_less)
predicted_layernorm_less = layer_norm_wo_feaures_predictor.predict(layernorm_features_less)
predicted_embedding_less = embedding_wo_feaures_predictor.predict(embedding_features_less)

predicted_dense_less = sum(predicted_dense_less * (dataset_size / batch_size)) / 1000
predicted_attention_less = sum(predicted_attention_less * (dataset_size / batch_size)) / 1000
predicted_embedding_less = sum(predicted_embedding_less * (dataset_size / batch_size)) / 1000
predicted_layernorm_less = sum(predicted_layernorm_less * (dataset_size / batch_size)) / 1000

predicted_e2e_less = predicted_dense_less + predicted_attention_less + predicted_embedding_less + predicted_layernorm_less
predicted_e2e_less = predicted_e2e_less.item()

In [29]:
print(f"Error in prediction with all features for BERT: {abs(predicted_e2e - time_for_one_epoch) / time_for_one_epoch * 100:.2f}%")
print(f'Predicted time for one epoch will all features for BERT: {predicted_e2e:.2f} seconds')
print(f'Actual time for one epoch: {time_for_one_epoch:.2f} seconds')

Error in prediction with all features for BERT: 62.97%
Predicted time for one epoch will all features for BERT: 3.04 seconds
Actual time for one epoch: 8.22 seconds


In [30]:
print(f"Error in prediction with raw features for BERT: {abs(predicted_e2e_less - time_for_one_epoch) / time_for_one_epoch * 100:.2f}%")
print(f'Predicted time for one epoch with raw features for BERT: {predicted_e2e_less:.2f} seconds')
print(f'Actual time for one epoch: {time_for_one_epoch:.2f} seconds')

Error in prediction with raw features for BERT: 64.07%
Predicted time for one epoch with raw features for BERT: 2.95 seconds
Actual time for one epoch: 8.22 seconds
